<a href="https://colab.research.google.com/github/KucharskiR/data-science/blob/main/trained/001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Sieć neuronowa LSTM**

Budowa sieci neuronowej LSTM

#### **Spis treści:**
1. [Import bibliotek](#a0)
2. [Przygotowanie danych](#a1)
3. [Konfiguracja LSTM layer](#a2)
4. [Główny model](#a3)
5. [Klasyfikacja](#a4)
6. [Przykład budowy modelu z kursu](#a5)
7. [Ocena modelu + wykresy](#a6)
8. [Predykcja na podstawie modelu](#a7)
9. [Zip file](#a8)
10. [Extract .tar.gz](#a9)

### <a name='a0'></a> Import bibliotek

In [1]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from keras.initializers import TruncatedNormal
from keras.models import save_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import os

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
# sns.set()   <--- seaborn
print(np.__version__)

1.25.2


### <a name='a1'></a> Przygotowanie danych

In [256]:
# Epochs
epochs = 5

In [257]:
# Assuming there are three features in your data
num_labels = 2
timestepsPerSample = 150
batch = 32

In [4]:
# Replace 'your_file.csv' with the actual file path
file_features = './1_150x9/1_150x9f.csv'
file_labels = './1_150x9/1_150x9l.csv'

# Specify the CSV file name
csv_file_name = 'my_data1.csv'

# Read the .csv file and create an array
data_strings = np.genfromtxt(file_features, delimiter=';')
labels_strings = np.genfromtxt(file_labels,delimiter=';')

In [258]:
# Wycinanie wybranych kolumn
# data_s = data_strings[:,[0,2,3,4,5,6,7,8]]
data_s = data_strings[:,[0,4,5,8]]
num_features = data_s.shape[1]
print(data_strings[:3])
print(data_s[:3])

[[-0.07371   0.001137  0.01742  -0.0145   -0.06481   0.08029  -0.11523   0.10792   0.989   ]
 [-0.17048   0.007527  0.019168 -0.01685  -0.10209  -0.44286  -0.18144   0.01512   0.94915 ]
 [-0.17888   0.007856  0.018207 -0.01051  -0.09147  -0.71242  -0.25967  -0.04955   0.76432 ]]
[[-0.07371 -0.06481  0.08029  0.989  ]
 [-0.17048 -0.10209 -0.44286  0.94915]
 [-0.17888 -0.09147 -0.71242  0.76432]]


In [259]:
# Convert from strings to float and int
X = data_s.astype(float).reshape((-1,timestepsPerSample,num_features))
Y = labels_strings.astype(float).reshape((-1,3))
print(X.shape)
print(Y.shape)

(33300, 150, 4)
(33300, 3)


In [348]:
# Modification from imported to new size
X_mod = X[1000:11000,130:]
Y_mod = Y[1000:11000]
timestepsPerSample = X_mod.shape[1]
print(X_mod.shape)
print(Y_mod.shape)
print(X_mod[:1])
# print(X[:1])

(10000, 20, 4)
(10000, 3)
[[[ 0.27949  0.14847  0.79856  0.99734]
  [ 0.26975  0.12186  0.74     0.99837]
  [ 0.24924  0.17163  0.52381  0.99875]
  [ 0.25571  0.15738  0.22449  0.99897]
  [ 0.23283  0.18952 -0.02128  0.99898]
  [ 0.17516  0.112   -0.22449  0.99879]
  [ 0.05792  0.06112 -0.6      0.99816]
  [ 0.08473  0.10709 -0.6      0.99754]
  [ 0.18472  0.14816 -0.25641  0.99763]
  [ 0.07894  0.08105 -0.02778  0.99711]
  [ 0.11305  0.10543  0.24242  0.99692]
  [ 0.08073  0.14804 -0.0303   0.99657]
  [-0.02131  0.04477 -0.28358  0.99528]
  [-0.11164  0.03727 -0.62963  0.99219]
  [-0.1537  -0.0509  -0.87097  0.98657]
  [-0.21008 -0.14329 -0.87387  0.97478]
  [-0.21008 -0.14598 -0.84211  0.95592]
  [-0.14679 -0.14043 -0.65766  0.93689]
  [-0.18358 -0.1647  -0.46667  0.90984]
  [-0.17    -0.17851 -0.25714  0.88401]]]


In [376]:
# splitting the dataset 75% for training and 25% testing
# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
x_train, x_test, Y_train, Y_test = train_test_split(X_mod,Y_mod, test_size=0.15, shuffle=False)
print(x_train.shape)
# print(x_train[:5])

(8500, 20, 4)


In [377]:
y_train = Y_train[:, 0:2]
y_test = Y_test[:, 0:2]
print(y_train[:2])
print(y_test[:2])

[[0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]]


### <a name='a4'></a> Konfiguracja LSTM layer

In [378]:
LstmLayer = LSTM(
    units=200,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.2,
    recurrent_dropout=0.0,
    seed=None,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    unroll=False,
    input_shape=(x_train.shape[1],x_train.shape[2])
)

### <a name='a3'></a> Główny model

In [379]:
# Optimizer decay
num_samples = x_train.shape[0]
STEPS_PER_EPOCH = num_samples/batch
num_samples
print(f"Timesteps: {timestepsPerSample}")
print(f"Num Samples: {num_samples}")
print(f"Num features: {num_features}")

Timesteps: 20
Num Samples: 8500
Num features: 4


In [380]:
model = Sequential() # initializing model

# input layer and LSTM layer with 50 neurons
# lst.add(LSTM(units=300, return_sequences=False, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(LstmLayer)
model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
model.add(Dense(20, activation='relu'))
# outpute layer with sigmoid activation
model.add(Dense(num_labels, activation='sigmoid'))

# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*100,
  decay_rate=0.9,
  staircase=False)

def get_optimizer():
  # return tf.keras.optimizers.Adam(learning_rate=0.001)
  return tf.keras.optimizers.Adam(learning_rate= lr_schedule)

optimizer = get_optimizer()

# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 200)               164000    
                                                                 
 dense_56 (Dense)            (None, 100)               20100     
                                                                 
 dense_57 (Dense)            (None, 20)                2020      
                                                                 
 dense_58 (Dense)            (None, 2)                 42        
                                                                 
Total params: 186162 (727.20 KB)
Trainable params: 186162 (727.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training

In [381]:
# training the model on training dataset
# history = lst.fit(x_train, y_train, epochs=epochs, batch_size=batch,validation_split=0.2)
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch, validation_data=(x_test, y_test))

# Save model
model.save("1_150x_trained.h5", overwrite=True, save_format='h5')

# predicting target attribute on testing dataset
predict = model.predict(x_test)
# predict = np.argmax(lst.predict(x_test), axis=-1)

Epoch 1/5
266/266 [==============================] - 16s 46ms/step - loss: 0.6232 - accuracy: 0.6629 - val_loss: 0.5692 - val_accuracy: 0.7120
Epoch 2/5
266/266 [==============================] - 13s 49ms/step - loss: 0.5994 - accuracy: 0.6816 - val_loss: 0.5700 - val_accuracy: 0.7093
Epoch 3/5
266/266 [==============================] - 13s 49ms/step - loss: 0.5939 - accuracy: 0.6889 - val_loss: 0.5570 - val_accuracy: 0.7120
Epoch 4/5
266/266 [==============================] - 13s 49ms/step - loss: 0.5924 - accuracy: 0.6864 - val_loss: 0.5625 - val_accuracy: 0.7120
Epoch 5/5
266/266 [==============================] - 13s 48ms/step - loss: 0.5851 - accuracy: 0.6960 - val_loss: 0.5723 - val_accuracy: 0.7140


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



47/47 [==============================] - 1s 14ms/step


In [382]:
# Set print options to suppress scientific notation
np.set_printoptions(suppress=True)

# Concatenate arrays
result = np.hstack((predict, y_test))
print(result)

test_results = model.evaluate(x_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

# # Create a DataFrame from the 2D array
# df = pd.DataFrame(predict, columns=['Column1', 'Column2'])

# # Export the DataFrame to CSV with semicolon as the delimiter and avoiding scientific notation
# # df.to_csv(csv_file_name, sep=';', index=False, float_format='%.0f')
# df.to_csv(csv_file_name, sep=';', index=False)

# # Read the CSV file into a DataFrame without header
# df = pd.read_csv(csv_file_name, sep=';', header=None)

# # Drop the first row containing data
# df = df.iloc[1:]

# # Save the modified DataFrame back to the CSV file without header
# df.to_csv(csv_file_name, sep=';', index=False, header=False)

# print(f'CSV file name: {csv_file_name}.')



[[0.65295124054  0.349432885647 0.             1.            ]
 [0.688349425793 0.307092696428 0.             1.            ]
 [0.710048615932 0.284625679255 0.             1.            ]
 ...
 [0.415741801262 0.571114420891 0.             1.            ]
 [0.434298396111 0.551436603069 0.             1.            ]
 [0.398584574461 0.584887742996 0.             1.            ]]
47/47 [==============================] - 1s 14ms/step - loss: 0.5723 - accuracy: 0.7140
Test results - Loss: 0.5723249912261963 - Accuracy: 71.39999866485596%


### <a name='a4'></a> Predict i Klasyfikacja

In [383]:
predict = model.predict(x_test)
predict[:3]

47/47 [==============================] - 1s 13ms/step


array([[0.65295124, 0.3494329 ],
       [0.6883494 , 0.3070927 ],
       [0.7100486 , 0.28462568]], dtype=float32)

In [384]:
# np.info(predict)
# np.where(y_pred > threshold, 1,0)
predict_classes = predict
# predict_classes = np.argmax(predict_classes, axis=-1)
predict_classes = np.where(predict > 0.5, 1,0)
np.info(predict_classes)
predict_classes[:3]
# predict

class:  ndarray
shape:  (1500, 2)
strides:  (16, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x5b1933c13de0
byteorder:  little
byteswap:  False
type: int64


array([[1, 0],
       [1, 0],
       [1, 0]])

**epochs** - ile razy zestaw treningowy zostanie przetworzony przez model. Przy każdej iteracji optymalizator próbuje dopasować wagi, aby funkcja celu została zminimalizowana.

**batch_size** - liczba przykładów treningowych po której następuje aktualizacji wag

**validation_split** - procent danych użytych do walidacji

In [385]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics[:3]

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.623158,0.662941,0.569152,0.712000,0
1,0.599403,0.681647,0.570017,0.709333,1
2,0.593932,0.688941,0.557011,0.712000,2


### <a name='a6'></a> Ocena modelu LSTM

In [386]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [387]:
# nie wiem co to za blok
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

In [388]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(test_acc)

0.7139999866485596


### <a name='a7'></a> Graf zielono czerwony

In [389]:
# Połączenie dwóch tablic
concat = np.hstack((predict_classes, Y_test[:,2:]))
print(concat.shape)
print(concat[:3])

(1500, 3)
[[ 1.    0.   87.53]
 [ 1.    0.   87.52]
 [ 1.    0.   87.55]]


In [390]:
import plotly.express as px
import pandas as pd

# Tworzenie Date Frame
df = pd.DataFrame(concat, columns=['Sell', 'Buy', 'Price'])
df.head(5)

,Sell,Buy,Price
0,1.000000,0.000000,87.530000
1,1.000000,0.000000,87.520000
2,1.000000,0.000000,87.550000
3,1.000000,0.000000,87.510000
4,1.000000,0.000000,87.530000


In [391]:
import plotly.graph_objects as go
import pandas as pd

# Tworzenie wykresu liniowego
fig = go.Figure()

start = 1
end = start + 1480

# Dodawanie linii do wykresu
for i in range(start, len(df[:end])):
    color = 'green' if df.at[i, 'Buy'] == 1 else 'red'
    fig.add_trace(go.Scatter(x=[i-1, i], y=[df.at[i-1, 'Price'], df.at[i, 'Price']], line=dict(color=color), showlegend=False))

# Ustawienia osi i tytuł
fig.update_layout(
    xaxis_title='Indeks',
    yaxis_title='Close',
    title='Wykres liniowy z kolorami'
)

# Wyświetlanie wykresu
fig.show()

In [393]:
# Profit
sum = 0;
for i in range(0, len(df)):
  if ((i-1) > 0):
    if df.at[i,'Sell'] == 1:
      sum += (df.at[i,'Price'] - df.at[i-1,'Price'])*(-1)
    elif df.at[i,'Buy'] == 1:
      sum += (df.at[i,'Price'] - df.at[i-1,'Price'])
    if df.at[i,'Sell'] != df.at[i-1,'Sell']:
      sum -= 0.03

print(sum)

1.4600000000001134


### <a name='a7'></a> Predykcja na podstawie modelu:



1.   **model.evaluate(y_true, y_pred)** - pozwala obliczyć metryki modelu
2.   **model.predict_classes()** - pozwala zwrócić odpowiednio przewidziane klasy
3.   **model.predict_proba(), model.predict()** - pozwala zwrócić prawdopodobieństwo danej klasy





In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
# predictions_cls = model.predict_classes(X_test)
predictions_cls = np.argmax(model.predict(x_test), axis=-1)
predictions_cls

In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
model = tf.keras.models.load_model('lstm_Model.keras')
model.summary()

### <a name='a5'></a> Przykład budowy modelu z kursu

In [ ]:
# model = Sequential()
# model.add(Flatten(input_shape=(28, 28)))
# model.add(Dense(units=128, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))

# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.summary()
model = Sequential()
model.add(LstmLayer)
model.add(Dense(units=2, activation='softmax')) # <----- output layer

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

### <a name='a8'></a> Zip file

In [ ]:
from zipfile import ZipFile

zip = ZipFile('my_python_files.zip','w')
zip.write('lstm_Model.keras')

### <a name='a9'></a> Extract tar gz

In [2]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open('1_150x9.tar.gz')

# extracting a specific file
file.extractall(path='./1_150x9/')

file.close()